In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd 
import numpy as np 
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

os.chdir("/content/drive/MyDrive/HAM10000")
!ls

HAM10000_images_augmented	 HAM10000_metadata.csv
HAM10000_images_part_1		 HAM10000_segmentations_lesion_tschandl
HAM10000_images_part_2		 training_data.csv
HAM10000_metadata_augmented.csv  training_data.npy


In [ ]:
#get metadata
mdata = pd.read_csv("/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv")
mdata

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern


In [ ]:
#for age 
nancount = 0 
for i in mdata["age"]:
  if pd.isna(i):
    print("nan")
    nancount += 1 
print(nancount)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
57


In [ ]:
#for sex 
nancount = 0 
for i in mdata["sex"]:
  if pd.isna(i):
    print("nan")
    nancount += 1 
print(nancount)

0


In [ ]:
#transform the function according to the pytorch docs
from torchvision import transforms
from PIL import Image
img_size = 224
preprocess = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    #perhaps we can also consider increasingt the contrast of the image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# preprocess2 = preprocess = transforms.Compose([
    
# ])
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

In [ ]:
from pyparsing.helpers import identbodychars
class DermClassHelper(): 
  #make images 50 x 50 to start 
  mdata = pd.read_csv("/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv").append(pd.read_csv("/content/drive/MyDrive/HAM10000/HAM10000_metadata_augmented.csv"))
  IMG_SIZE = 224
  labels = {}
  img_map = {}
  #training data 
  training_data = []
  # track balance 
  balance = {}
  #relavent directories 
  sds = ["/HAM10000_images_part_1/", "/HAM10000_images_part_2/", "/HAM10000_images_augmented/"]
  root = "/content/drive/MyDrive/HAM10000"
  #map file name to age and sex 
  info_map = {}

  def __init__(self):
    #assign labels
    self.make_labels()
    #make the map
    self.make_image_map()
    #make info map 
    self.make_info_map()
    print(self.labels, self.info_map)

  """ Assign numbers to the diagnosis types"""
  def make_labels(self):
    i = 0 
    for d in self.mdata["dx"].unique():
      self.labels[d] = i 
      i += 1
  """ Assign to each image the corresponding label """
  def make_image_map(self):
    for im, dx in zip(list(self.mdata["image_id"]), list(self.mdata["dx"])):
      self.img_map[im] = self.labels[dx]

  def make_info_map(self):
    for imname, group in tqdm(self.mdata.groupby("image_id")):
      sex = None
      if list(group["sex"])[0] == "male":
        sex = -1
      else:
        sex = 1
      self.info_map[imname] = [list(group["age"])[0], sex]

  def make_training_data(self):
    for sd in self.sds:
      for f in tqdm(os.listdir(self.root+sd)):
        # extract the image id from the file path
        id = f.split(".")[0].strip(" ").split(" ")[0]
        if self.img_map[id] not in self.balance:
          self.balance[self.img_map[id]] = 0
        self.balance[self.img_map[id]] += 1
    print(self.balance)

In [ ]:
dch = DermClassHelper()
dch.make_training_data()
print(dch.balance)

100%|██████████| 10934/10934 [00:01<00:00, 8706.09it/s]


{'bkl': 0, 'nv': 1, 'df': 2, 'mel': 3, 'vasc': 4, 'bcc': 5, 'akiec': 6} {'ISIC_0024306': [45.0, -1], 'ISIC_0024307': [50.0, -1], 'ISIC_0024308': [55.0, 1], 'ISIC_0024309': [40.0, -1], 'ISIC_0024310': [60.0, -1], 'ISIC_0024311': [75.0, 1], 'ISIC_0024312': [75.0, -1], 'ISIC_0024313': [50.0, 1], 'ISIC_0024314': [75.0, -1], 'ISIC_0024315': [55.0, -1], 'ISIC_0024316': [55.0, -1], 'ISIC_0024317': [35.0, 1], 'ISIC_0024318': [65.0, 1], 'ISIC_00243181': [65.0, 1], 'ISIC_00243182': [65.0, 1], 'ISIC_00243183': [65.0, 1], 'ISIC_0024319': [55.0, -1], 'ISIC_0024320': [65.0, -1], 'ISIC_0024321': [35.0, -1], 'ISIC_0024322': [65.0, 1], 'ISIC_0024323': [50.0, -1], 'ISIC_0024324': [85.0, -1], 'ISIC_0024325': [40.0, 1], 'ISIC_0024326': [75.0, -1], 'ISIC_0024327': [50.0, 1], 'ISIC_0024328': [55.0, -1], 'ISIC_0024329': [75.0, 1], 'ISIC_0024330': [40.0, -1], 'ISIC_00243301': [40.0, -1], 'ISIC_00243302': [40.0, -1], 'ISIC_00243303': [40.0, -1], 'ISIC_0024331': [65.0, -1], 'ISIC_0024332': [60.0, -1], 'ISIC_002

100%|██████████| 3006/3006 [00:00<00:00, 459662.32it/s]

{1: 6806, 5: 1023, 3: 1634, 0: 1612, 6: 832, 2: 616, 4: 645}
{1: 6806, 5: 1023, 3: 1634, 0: 1612, 6: 832, 2: 616, 4: 645}


In [ ]:
dch.mdata

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,Unnamed: 0
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,NaN
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,NaN
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,NaN
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,NaN
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,NaN
...,...,...,...,...,...,...,...,...,...
3001,HAM_0000769,ISIC_00292791,akiec,histo,80.0,male,lower extremity,Augmented,0.0
3002,HAM_0002646,ISIC_00285581,akiec,histo,75.0,male,face,Augmented,0.0
3003,HAM_0004568,ISIC_00275291,akiec,histo,50.0,male,upper extremity,Augmented,0.0
3004,HAM_0004568,ISIC_00244631,akiec,histo,50.0,male,upper extremity,Augmented,0.0


In [ ]:
nancount = 0 
for i in dch.mdata["age"]:
  if pd.isna(i):
    print("nan")
    nancount += 1 
print(nancount)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
57


In [ ]:
#for sex 
nancount = 0 
for i in dch.mdata["sex"]:
  if pd.isna(i):
    print("nan")
    nancount += 1 
print(nancount)

0


In [ ]:
len(dch.mdata)
#drop the rows with nan in age
dch.mdata = dch.mdata[dch.mdata['age'].notna()]
len(dch.mdata)

12964

In [ ]:
from skimage import io

In [ ]:
#write a different data loader class 
from torch.utils.data import DataLoader
class DermClass():
  def __init__(self, helper, csv_file, root_dir, sds, transform1=None):
    self.derm_frame = pd.read_csv(csv_file)
    #drop nandata 
    self.derm_frame = self.derm_frame[self.derm_frame['age'].notna()]
    self.root_dir = root_dir
    self.sds = sds 
    self.transform1 = transform1
    #pass in derm class helper here
    self.helper = helper

  def __len__(self):
    return len(self.derm_frame)

  def __getitem__(self, index):
    #1 is the image file col 
    filename = self.derm_frame.iloc[index, 1]
    img_path = None
    #find the image path  
    if os.path.exists(str(self.root_dir+self.sds[0]+filename+".jpg")):
      img_path = str(self.root_dir+self.sds[0]+filename+".jpg")
    elif os.path.exists(str(self.root_dir+self.sds[1]+filename+".jpg")):
      img_path = str(self.root_dir+self.sds[1]+filename+".jpg")

    #get the image using PIL 
    img = Image.open(img_path)
    # y_label = torch.tensor(np.eye(len(self.helper.labels))[self.helper.img_map[filename]]) #this is good for adam optim 
    y_label = torch.tensor(int(self.helper.img_map[filename]))
    # print(pd.isna(self.helper.info_map[filename][0]))
    info = torch.tensor([float(self.helper.info_map[filename][0]*.1), float(self.helper.info_map[filename][1])])
    # info = torch.tensor([self.helper.info_map[filename][1]])

    if self.transform1: 
      image = self.transform1(img)
    return (image, y_label, info)

In [ ]:
# g = DermClass(helper=dch, csv_file="/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv", root_dir="/content/drive/MyDrive/HAM10000", sds=["/HAM10000_images_part_1/", "/HAM10000_images_part_2/", "/HAM10000_images_augmented/"], transform1=preprocess)
# g.__getitem__(33)

In [ ]:
# i = 0 
# g = DermClass(helper=dch, csv_file="/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv", root_dir="/content/drive/MyDrive/HAM10000", sds=["/HAM10000_images_part_1/", "/HAM10000_images_part_2/", "/HAM10000_images_augmented/"], transform1=preprocess)
# while True: 
#   g.__getitem__(i)
#   i += 1


In [ ]:
class AugmentedDermClass():
  def __init__(self, helper, csv_file, root_dir, sd, transform1=None):
    self.derm_frame = pd.read_csv(csv_file)
    #drop nan age data 
    self.derm_frame = self.derm_frame[self.derm_frame['age'].notna()]
    self.root_dir = root_dir
    self.sd = sd
    self.transform1 = transform1
    #pass in derm class helper here
    self.helper = helper

  def __len__(self):
    return len(self.derm_frame)

  def __getitem__(self, index):
    #1 is the image file col 
    filename = self.derm_frame.iloc[index, 1]
    # dataset = self.derm_frame.iloc[index, 7]
    img_path = None
    #find the image path  
    if os.path.exists(str(self.root_dir+self.sd+filename+".jpg")):
      img_path = str(self.root_dir+self.sd+filename+".jpg")

    #get the image using PIL 
    img = Image.open(img_path)
    # y_label = torch.tensor(np.eye(len(self.helper.labels))[self.helper.img_map[filename]]) #this is good for adam optim 
    y_label = torch.tensor(int(self.helper.img_map[filename]))
    # pd.isna(self.helper.info_map[filename][0])
    info = torch.tensor([float(self.helper.info_map[filename][0]*.1), float(self.helper.info_map[filename][1])])
    # info = torch.tensor([self.helper.info_map[filename][1]])
    if self.transform1: 
      image = self.transform1(img)
    return (image, y_label, info)

In [ ]:
# i = 0 
# g = AugmentedDermClass(helper=dch, csv_file="/content/drive/MyDrive/HAM10000/HAM10000_metadata_augmented.csv", root_dir="/content/drive/MyDrive/HAM10000", sd = "/HAM10000_images_augmented/", transform1=preprocess)
# while True: 
#   g.__getitem__(i)
#   i += 1


In [ ]:
# g = DermClass(helper=dch, csv_files=["/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv", "/content/drive/MyDrive/HAM10000/HAM10000_metadata_augmented.csv"], root_dir="/content/drive/MyDrive/HAM10000", sds=["/HAM10000_images_part_1/", "/HAM10000_images_part_2/", "/HAM10000_images_augmented/"], transform1=preprocess)
# g.__getitem__(11000)


In [ ]:
#hyperparams
in_channel = 3
num_classes = 7
learning_rate = 1e-3
batch_size = 64
num_epochs =10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
dataset = DermClass(helper=dch, csv_file="/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv", root_dir="/content/drive/MyDrive/HAM10000", sds=["/HAM10000_images_part_1/", "/HAM10000_images_part_2/"], transform1=preprocess)

augmented_dataset = AugmentedDermClass(helper=dch, csv_file="/content/drive/MyDrive/HAM10000/HAM10000_metadata_augmented.csv", root_dir="/content/drive/MyDrive/HAM10000", sd= "/HAM10000_images_augmented/", transform1=preprocess)

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*.75), int(len(dataset)*.25)+1])

In [ ]:
#here, we need to find a way to add augmented images to training and exclude them from testing 
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*.75), int(len(dataset)*.25)+1])

#add the augmented data to the train set
train_set = torch.utils.data.ConcatDataset([train_set, augmented_dataset])


train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True)

In [ ]:
len(train_loader)

164

In [ ]:
# dataset.__getitem__(0)

In [ ]:
#import pretrained 
import torch
# resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18',  pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch.nn.functional as F 

#null class that will be useful for us 
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
  
class ResNetFeature(nn.Module):
  def __init__(self, num_classes=7):
    super(ResNetFeature, self).__init__()
    #define the cnn component here to be resnet 
    self.cnn = resnet
    #make the last fc layer just the identity 
    self.cnn.fc = nn.Linear(2048, 100)
    self.enable_finetuning() #enable the finetuning

    #add the additional fc layers
    # self.fc1 = nn.Linear(2, 100) #this is where we take sex/gender and explode it to 100 dimensions
    self.fc2 = nn.Sequential(
                        nn.Linear(102, 100),
                        nn.ReLU(), 
                        nn.Linear(100, 100), 
                        nn.ReLU(), 
                        nn.Dropout(p=.5), 
                        nn.Linear(100, 7))                         
  def enable_finetuning(self):
    #perform various changes to the layers required for finetuning
    thresh = 5 #should be 6
    ct = 0
    #here we freeze up to and including the 6th layer
    for child in self.cnn.children():
      if ct <= thresh:
        for param in child.parameters():
            param.requires_grad = False
      print(child, ct)
      ct += 1

  def forward(self, x, y):
    x = self.cnn(x)
    y = F.normalize(y)
    y = F.relu(self.fc1(y)) # 100 
    x = torch.cat((x, y), dim=1) # 102
    x = self.fc2(x) # 7
    return x

In [ ]:
model = ResNetFeature()
model.train()
model.to(device)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 0
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 1
ReLU(inplace=True) 2
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False) 3
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

ResNetFeature(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [ ]:
# Loss and optimizer
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.


criterion = nn.CrossEntropyLoss()
# criterion = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean') #experimenting with focal loss 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.1, patience=5, verbose=True)
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets, info) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        info = info.to(device=device)
        targets = targets.to(device=device)
        # forward
        scores = model(data, info)
        loss = criterion(scores, targets)
        print("Batch: %d. Loss: %f" %(batch_idx, loss))

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()
        
        #clip gradient 
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)

        # gradient descent or adam step
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")


1it [00:00,  1.18it/s]

Batch: 0. Loss: 1.950230


2it [00:01,  1.21it/s]

Batch: 1. Loss: 1.650151


3it [00:02,  1.19it/s]

...

Batch: 161. Loss: 0.173798


163it [02:14,  1.22it/s]

Batch: 162. Loss: 0.365741


164it [02:14,  1.22it/s]

Batch: 163. Loss: 0.155422
Cost at epoch 9 is 0.2485168587143828


In [ ]:

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    correct = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
    total = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
    with torch.no_grad():
        for x, y, info in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)
            info = info.to(device=device)
            scores = model(x, info)
            
            _, predictions = scores.max(1)
            for i,j in zip(predictions, y):
              if i.item() == j.item():
                correct[i.item()] +=1
              total[j.item()] += 1
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
              f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
          )
        #find the accuracies for each class 
        return correct, total

    model.train()


# print("Checking accuracy on Training Set")
# check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
correct, total = check_accuracy(test_loader, model)

for k in correct:
  print("for class %d, the accuracy is: %f" %(k, correct[k]/total[k]))

Checking accuracy on Test Set


100%|██████████| 39/39 [00:33<00:00,  1.18it/s]

Got 1932 / 2490 with accuracy 77.59
for class 0, the accuracy is: 0.380952
for class 1, the accuracy is: 0.969091
for class 2, the accuracy is: 0.440000
for class 3, the accuracy is: 0.318519
for class 4, the accuracy is: 0.902439
for class 5, the accuracy is: 0.588235
for class 6, the accuracy is: 0.157895


In [ ]:
print(correct, total)

{0: 104, 1: 1599, 2: 11, 3: 86, 4: 37, 5: 80, 6: 15} {0: 273, 1: 1650, 2: 25, 3: 270, 4: 41, 5: 136, 6: 95}


In [ ]:
from google.colab import runtime
runtime.unassign()